In [106]:
import pandas as pd
import numpy as np

# for plotting
import matplotlib.pyplot as plt

# for processing
from tqdm import tqdm
from sklearn.model_selection import train_test_split, GridSearchCV

# for scoring
from sklearn.metrics import classification_report

# for modeling
import torch
from transformers import AutoTokenizer, AutoModel, TrainingArguments, Trainer
from datasets import Dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

In [77]:
LABELS = (
    'athletics',
    'autosport',
    'basketball',
    'boardgames',
    'esport',
    'extreme',
    'football',
    'hockey',
    'martial_arts',
    'motosport',
    'tennis',
    'volleyball',
    'winter_sport',
)

TRAIN_CSV = '~/Downloads/train.csv'
TEST_CSV = '~/Downloads/test.csv'

In [10]:
test_data = pd.read_csv(TEST_CSV)
test_data

,oid,text
0,749208109,СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПК...
1,452466036,Естественное восстановление после тяжелой трен...
2,161038103,Тема нарядов продолжается Одна из британских ж...
3,663621910,Привет Избранный. Ты спрашиваешь себя ЧТО здес...
4,566255305,КОРОЛЬ ПЯТИСОТНИКОВ В ДЕЛЕ Андрей Рублев успеш...
...,...,...
26255,169728316,Выиграй коллекционный пазл по Wortokenoid of W...
26256,279369911,Волейбол от первого лица Егора Пупынина переко...
26257,600699419,Вы были когда нибудь на свидании где вам задав...
26258,560223506,ТОП 20 самых эффективных общефизических упражн...


In [11]:
train_data = pd.read_csv(TRAIN_CSV)
train_data

,oid,category,text
0,365271984,winter_sport,Волшебные фото Виктория Поплавская ЕвгенияМедв...
1,503385563,extreme,Возвращение в подземелье Треша 33 Эйфория тупо...
2,146016084,football,Лучшие чешские вратари – Доминик Доминатор Гаш...
3,933865449,boardgames,Rtokenoid Warhammer40k валрак решил нас подкор...
4,713550145,hockey,Шестеркин затаскивает Рейнджерс в финал Восточ...
...,...,...,...
38735,910636962,autosport,8 битная буря снова накрыла пикселями автомоби...
38736,669736851,autosport,Ира Сидоркова объясняет как сказалась на ее ма...
38737,558919241,tennis,24 я ракетка мира хорват Марин Чилич обыграл и...
38738,776944963,volleyball,Стал известен календарь мужской сборной России...


In [42]:
train_data['text'][train_data['category'] == 'esport']

7        Какой т3 игрок на данный момент перспективнее ...
14       Гвардиола об 1 0 с Атлетико Очень равный матч ...
21       Сегодня подведем итоги трех конкурсов на одном...
27       Ровно шесть лет назад s1mptokenoid присоединил...
40       Btokenoid на подкасте EsportsDotokenoid Я нико...
                               ...                        
38695    Барса даст Левандовскому двухлетний или трехле...
38697    ️АЛЕКСАНДР КОКОРИН ПРОПУСТИТ МАТЧ СО СПЕЦИЕЙ И...
38707    В группе один Администратор Добавляй в друзья ...
38717    Счастливчики которые уносят Кассадина 33 За пр...
38732    etokenoid Я был самым большим фанатом shrotoke...
Name: text, Length: 2990, dtype: object

In [141]:
train_data[train_data['category'] == 'martial_arts']

,oid,category,text
24,300500058,martial_arts,Как выглядел мир когда последний раз дрался Ди...
55,652133165,martial_arts,Обувная ложка меч Kessak подарок для настоящег...
97,545572382,martial_arts,Как заменить привычные чипсы? Отведайте сушки ...
102,827524142,martial_arts,Меня просто спровоцировали а те кто пишет в мо...
104,102339046,martial_arts,AMC FNG Амиржанов Гаджимурад Денисов Сергей 23...
...,...,...,...
38665,352307844,martial_arts,Они опять в Таиланде Кто помнит предыдущие фот...
38683,114699964,martial_arts,Дана Уайт сообщил когда и с кем будет драться ...
38689,271219366,martial_arts,Генри Сехудо считает что Хамзат Чимаев эмоцион...
38691,545572382,martial_arts,Детский набор для творчества с мольбертом 176 ...


In [132]:
tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-conversational', use_fast=True)
model = AutoModel.from_pretrained('DeepPavlov/rubert-base-cased-conversational', num_labels=len(LABELS))

def preprocess_function(data):
    return tokenizer(data['text'], data['category'], max_length=512, truncation=True)

# X_train, X_test, y_train, y_test = train_test_split(train_data['text'], train_data['category'], test_size=0.2, random_state=42)
encoded_dataset = Dataset.from_pandas(train_data).map(preprocess_function)
small_train_dataset = encoded_dataset.shuffle(seed=42).select(range(1000))
small_test_dataset = encoded_dataset.shuffle(seed=42).select(range(1000))

loading configuration file config.json from cache at /Users/nikita.boyarskikh/.cache/huggingface/hub/models--DeepPavlov--rubert-base-cased-conversational/snapshots/645946ce91842a52eaacb2705c77e59194145ffa/config.json
Model config BertConfig {
  "_name_or_path": "DeepPavlov/rubert-base-cased-conversational",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.

  0%|          | 0/38740 [00:00<?, ?ex/s]

In [135]:
batch_size = 100
metric_name = 'cross-entropy'
args = TrainingArguments(
    f"rubert-base-cased-conversational-finetuned",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=False,
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return {
        metric_name: classification_report(predictions[:, 0], labels),
    }

trainer = Trainer(
    model,
    args,
    train_dataset=small_train_dataset,
    eval_dataset=small_test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()
trainer.evaluate()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 1000
  Num Epochs = 5
  Instantaneous batch size per device = 100
  Total train batch size (w. parallel, distributed & accumulation) = 100
  Gradient Accumulation steps = 1
  Total optimization steps = 50
  Number of trainable parameters = 177853440
The following columns in the training set don't have a corresponding argument in `BertModel.forward` and have been ignored: category, oid, text. If category, oid, text are not expected by `BertModel.forward`,  you can safely ignore this message.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text f

ValueError: The model did not return a loss from the inputs, only the following keys: last_hidden_state,pooler_output. For reference, the inputs it received are input_ids,token_type_ids,attention_mask.

In [ ]:
print(1)

In [ ]:
best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")
best_run.hyperparameters

In [59]:
rf_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', RandomForestClassifier()),
])
sgd_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('sgd_clf', SGDClassifier(random_state=42, loss='log_loss')),
])
knb_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('knb_clf', KNeighborsClassifier(n_neighbors=10)),
])
sgd_clf.fit(train_data['text'], train_data['category'])
# print(classification_report(predicted_sgd, train_data['category'][0:1]))

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('sgd_clf', SGDClassifier(loss='log_loss', random_state=42))])

In [86]:
# predicted_sgd = sgd_clf.predict(test_data['text'][0:10])
# for text, label in zip(test_data['text'][0:10], predicted_sgd):
#     print(text)
#     print(label)
#     print()
predicted_sgd_proba = sgd_clf.predict_proba(test_data['text'])
predicted_sgd = sgd_clf.predict(test_data['text'])

In [90]:
res = test_data.copy()
res['predicted_sgd'] = predicted_sgd
res['sgd_proba'] = predicted_sgd_proba.max(axis=1)
res = res.sort_values('sgd_proba')

In [137]:
for text, cat in zip(res['text'][-10:], res['predicted_sgd'][-10:]):
    print(cat, '\t', text)

athletics 	 Бег мотивация 33 Смотри вдохнавляйся беги 33 rtokenoid бег спорт забег всембег rtokenoid parkrtokenoid sport легкаяАтлетика
athletics 	 Как улучшить бег? Увеличте скорость 33 rtokenoid легкаяАтлетика спорт забег зож rtokenoid
athletics 	 Всем бег и легких ног 33 Бегите на позитиве друзья 33 rtokenoid rtokenoid бег легкаяАтлетика спорт всембег забегРФ забег rtokenoid
athletics 	 Всем бег и легких ног rtokenoid rtokenoid спорт мотивация бег забег пробежка бегун всембег беганутые
athletics 	 БЕГ ЭТО РАДОСТЬ Беги на позитиве 33. . rtokenoid. . . . . rtokenoid rtokenoid parkrtokenoid бег спорт забег всембег наспорте беганутые ла легкаяАтлетика
athletics 	 ‍ ️ ‍ ️Всем бег и легких ног друзья rtokenoid rtokenoid rtokenoid бег марафон марафонец забег забегРФ спорт атлет
athletics 	 А кто ты на светофоре? rtokenoid rtokenoid бег спорт забег марафон марафонец зож наспорте легкаяАтлетика
athletics 	 ‍ ️ ‍ ️Всем бег и легких ног rtokenoid rtokenoid rtokenoid бег легкаяАтлетика спорт зо

/var/folders/fl/ys8k2g5s0sdfvq58n15mnjdr0000gq/T/ipykernel_89018/1655474595.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  for text, cat in zip(res['text'][-10:], res['predicted_sgd'][-10:]):
